In [3]:
import os
import shutil
import subprocess

def modify_input_file(temperature, simulation_type, h):
    with open("INPUT/input.dat", "r") as f:
        lines = f.readlines()

    # Cerca le righe da modificare e sostituisci solo quelle
    for i, line in enumerate(lines):
        if line.startswith("SIMULATION_TYPE"):
            lines[i] = f"SIMULATION_TYPE {simulation_type} 1.0 {h}\n"
        elif line.startswith("TEMP"):
            lines[i] = f"TEMP {temperature}\n"

    with open("INPUT/input.dat", "w") as f:
        f.writelines(lines)


def rename_output_files(temperature, simulation_type, h):
    output_folder = "OUTPUT"
    files_to_rename = ["total_energy.dat", "magnetization.dat", "specific_heat.dat", "susceptibility.dat"]
    
    for filename in os.listdir(output_folder):
        if filename in files_to_rename:
            old_filepath = os.path.join(output_folder, filename)
            new_filename = f"{filename.split('.')[0]}_{temperature}_{int(simulation_type)}_{h}.dat"
            new_filepath = os.path.join(output_folder, new_filename)
            os.rename(old_filepath, new_filepath)



def main():
    #esegui il codice tramite makefile
    subprocess.run(["make"],cwd="SOURCE")

    # Itera attraverso tutte le temperature e i tipi di simulazione
    for simulation_type in [2, 3]:
        for temperature in [2.0, 1.75, 1.50, 1.25, 1.0, 0.75, 0.50]:
            for h in [0, 0.02]:
                modify_input_file(temperature, simulation_type, h)
                subprocess.run(["./simulator.exe"], cwd="SOURCE")
                rename_output_files(temperature, simulation_type, h)

if __name__ == "__main__":
    main()


g++ -c system.cpp -o system.o -O3 --std=c++11
g++ system.o particle.o random.o NSL_SIMULATOR.o -o simulator.exe -larmadillo
